In [5]:
from langchain.output_parsers import DatetimeOutputParser, CommaSeparatedListOutputParser, PydanticOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

### Output Parser
   LangChain mein output parser ka istemal aapko LLM se milne wale output ko desired format mein convert karne ke liye hota hai.

## CommaSeparatedListOutputParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = gemini_api_key

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=gemini_api_key,  # Ensure this variable is defined correctly
    temperature=0.2,
    verbose=True,
)

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

# Create the prompt template correctly
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# Use LLMChain to create the chain
chain = LLMChain(
    prompt=prompt,
    llm=llm,
    output_parser=output_parser
)

# Invoke the chain
res = chain.invoke({"subject" : "ice cream flavors"})
print(res)


## DatetimeOutputParser

#### 1. `.get_format_instructions()`
- **Kya Karta Hai**: Yeh method aapko batata hai ke output ka format kya hoga. 
- **Example**: Agar aapne specify kiya hai ke aapko date chahiye is format mein: "Fri 10/02/2024", toh jab aap `.get_format_instructions()` call karenge, yeh yeh format aapko dikhayega.

#### 2. `.parse()`
- **Kya Karta Hai**: Yeh method model se mila output ko check karta hai aur usay format karta hai.
- **Example**: Agar model se output aata hai "2024-02-10 00:00:00" aur aap chahte hain ke yeh "Fri 10/02/2024" ban jaaye, toh aap is output ko `.parse()` method mein dete hain. Yeh method yeh check karega aur agar output valid nahi hai, toh error throw karega.

#### Summary:
- **`.get_format_instructions()`**: Aapko format ka idea deta hai.
- **`.parse()`**: Model ka output le kar use correct format mein badalta hai.


In [19]:
from langchain.output_parsers import DatetimeOutputParser, CommaSeparatedListOutputParser, PydanticOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

# Load the API key from .env file
gemini_api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = gemini_api_key

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=gemini_api_key,  # Corrected the variable usage
    temperature=0.2,
    verbose=True,
)

# # Example 1
date_time_parser = DatetimeOutputParser(Hformat="%a %d/%m/%Y")
# print(date_time_parser.get_format_instructions())

human_template = "{request}\n{format_instruction}"

prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(human_template)
 ])

# print(prompt)

formated_prompt = prompt.format_messages(
    request="When did Pakistan gain independence?",
    format_instruction=date_time_parser.get_format_instructions(),
)

res = llm.invoke(formated_prompt)
# Example 1
print("Response Content :\t", res.content)

# Example 2
print("Response Content Parse :\t", date_time_parser.parse(
    res.content
))
 

Response Content :	 1947-08-14T00:00:00.000000Z 

Response Content Parse :	 1947-08-14 00:00:00



### Kya Ho Raha Hai:

1. **Parser Setup**: 
   - Aap `JsonOutputParser` ka istemal kar rahe hain. Yeh ek tool hai jo model se milne wale jawab ko JSON format mein badalta hai.

2. **Pydantic Object**:
   - `Joke` ek class hai jo batati hai ke aapko data kaise chahiye. Jaise, `Joke` class mein ho sakta hai ke "joke" aur "punchline" ki fields hon.

### Instructions Ko Inject Karna:
- Jab aap `pydantic_object=Joke` dete hain, toh aap parser ko keh rahe hain ke:
  - "Mujhe yeh output chahiye, jo `Joke` class ke format mein ho."

### Iska Faida:
- Yeh ensure karta hai ke jab model se jawab aaye, toh woh us structure ke mutabiq ho. Agar nahi hua, toh parser error dega.

### Example:
- Sochiye ke aapne `Joke` class mein yeh define kiya hai:
  ```python
  class Joke(BaseModel):
      joke: str
      punchline: str
  ```

- Agar model se aisa jawab aaye:
  ```json
  {
      "joke": "Why did the chicken cross the road?",
      "punchline": "To get to the other side!"
  }
  ```
- Toh `JsonOutputParser` yeh check karega ke yeh structure sahi hai ya nahi. Agar sahi hai, toh aapko yeh data mil jayega. Agar nahi, toh error aayega.

### Summary:
- Aap parser ko `Joke` class dete hain taake yeh ensure ho ke output us format mein ho jo aap chahte hain. Is tarah aapko sahi aur structured data milta hai.


In [2]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

# Load the environment variables
load_dotenv()

# Get the API key from the environment
gemini_api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = gemini_api_key

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=gemini_api_key,
    temperature=0.2,
    verbose=True,
)

# Define the data structure (Joke model)
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

# Set up the JSON output parser
parser = JsonOutputParser(pydantic_object=Joke)



# # Create the prompt template
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# # Create the LLM chain
# chain = LLMChain(
#     prompt=prompt,
#     llm=llm,  # Pass the LLM model correctly here
#     output_parser=parser  # Correctly set the output parser here
# )

# # Query to get the joke
# joke_query = "Tell me a joke."

# # Invoke the chain with the query
# res = chain.invoke({"query": joke_query})

# # Print the result
# print(res)


type of  <class 'langchain_core.output_parsers.json.JsonOutputParser'>
